In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

apple을 입력받으면 pple!를 출력하는 RNN을 구현해볼 것이다. 어떤 의미가 있지는 않고, 그저 RNN의 동작을 이해하기 위함이다.

In [3]:
input_str = "apple"
label_str = "pple!"
char_vocab = sorted(list(set(input_str + label_str)))
vocab_size = len(char_vocab)
char_vocab, vocab_size

(['!', 'a', 'e', 'l', 'p'], 5)

입력은 원-핫 벡터를 사용할 것이므로 입력의 크기는 문자 집합의 크기여야 한다.

In [4]:
input_size = vocab_size
hidden_size = 5
output_size = 5
lr = 0.1

In [9]:
char_to_index = {c: i for i, c in enumerate(char_vocab)}
char_to_index

{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}

나중에 예측 결과를 문자 시퀀스로 보기 위해 index_to_char도 만든다.

In [26]:
index_to_char = {v: k for k, v in char_to_index.items()}
index_to_char

{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}

In [11]:
x_data = [char_to_index[c] for c in input_str]
y_data = [char_to_index[c] for c in label_str]

In [12]:
# 배치 차원 추가
x_data = [x_data]
y_data = [y_data]

In [20]:
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]
x_one_hot

[array([[0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0.]])]

In [14]:
X = torch.FloatTensor(x_one_hot)
y = torch.LongTensor(y_data)

C:\Users\jiweo\AppData\Local\Temp\ipykernel_34804\985899240.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:281.)
  X = torch.FloatTensor(x_one_hot)


In [15]:
print(f"훈련 데이터의 크기: {X.shape}")  # (batch_size, sequence_length, input_size)
print(f"레이블의 크기: {y.shape}")

훈련 데이터의 크기: torch.Size([1, 5, 5])
레이블의 크기: torch.Size([1, 5])


In [16]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size, bias=True)

    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [17]:
net = Net(input_size, hidden_size, output_size)

In [18]:
outputs = net(X)
outputs.shape

torch.Size([1, 5, 5])

In [22]:
outputs.view(-1, input_size).shape, y.view(-1).shape

(torch.Size([5, 5]), torch.Size([5]))

In [23]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=lr)

In [27]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, input_size), y.view(-1))
    loss.backward()
    optimizer.step()

    result = outputs.data.numpy().argmax(axis=2)
    result_str = "".join([index_to_char[c] for c in np.squeeze(result)])
    print(
        f"{i} loss: {loss.item()} prediction: {result} true Y: {y_data} prediction str: {result_str}"
    )

0 loss: 1.3604114055633545 prediction: [[4 4 4 4 4]] true Y: [[4, 4, 3, 2, 0]] prediction str: ppppp
1 loss: 1.253530502319336 prediction: [[4 4 4 4 4]] true Y: [[4, 4, 3, 2, 0]] prediction str: ppppp
2 loss: 1.0889469385147095 prediction: [[4 4 4 2 4]] true Y: [[4, 4, 3, 2, 0]] prediction str: pppep
3 loss: 0.8855255246162415 prediction: [[4 4 3 2 4]] true Y: [[4, 4, 3, 2, 0]] prediction str: pplep
4 loss: 0.7421501874923706 prediction: [[4 4 3 2 4]] true Y: [[4, 4, 3, 2, 0]] prediction str: pplep
5 loss: 0.623257040977478 prediction: [[4 4 3 2 4]] true Y: [[4, 4, 3, 2, 0]] prediction str: pplep
6 loss: 0.528544545173645 prediction: [[4 4 3 2 4]] true Y: [[4, 4, 3, 2, 0]] prediction str: pplep
7 loss: 0.4619673788547516 prediction: [[4 4 3 2 4]] true Y: [[4, 4, 3, 2, 0]] prediction str: pplep
8 loss: 0.4200817942619324 prediction: [[4 4 3 2 4]] true Y: [[4, 4, 3, 2, 0]] prediction str: pplep
9 loss: 0.3903277814388275 prediction: [[0 4 3 2 0]] true Y: [[4, 4, 3, 2, 0]] prediction str: